# What is a tokenizer?

A system that converts a text into a sequence of discreet symbols (tokens)


Tokenizer usually includes the following steps
- text normalization / clean up
- token splitting (rules on how text is split)
- vocabulary
- encoder (tokens -> integers)
- decoder (integers -> tokens)

here the implementation is for a BPE Tokenizer.

Reference:
- https://en.wikipedia.org/wiki/Byte-pair_encoding
- https://huggingface.co/learn/llm-course/en/chapter6/5



![Status](https://img.shields.io/badge/status-dev%20in%20progress-yellow)

#1. Set up

##1.1 loading the dataset

In [2]:
!wget -q https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
!wget -q https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt

!wget -q https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip -f owt_train.txt.gz
!wget -q https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_valid.txt.gz
!gunzip -f owt_valid.txt.gz

!ls -lh

total 14G
-rw-r--r-- 1 root root  12G Feb  7 22:11 owt_train.txt
-rw-r--r-- 1 root root 277M Feb  7 22:14 owt_valid.txt
drwxr-xr-x 1 root root 4.0K Jan 16 14:24 sample_data
-rw-r--r-- 1 root root 2.1G Feb  7 22:10 TinyStoriesV2-GPT4-train.txt
-rw-r--r-- 1 root root  22M Feb  7 22:10 TinyStoriesV2-GPT4-valid.txt


##1.2 overview of dataset

In [47]:
!echo "== TinyStories train =="; head -n 5 TinyStoriesV2-GPT4-train.txt #head -n 5 --> show me the first 5 lines of
!echo "== TinyStories valid =="; head -n 5 TinyStoriesV2-GPT4-valid.txt
!echo "== OWT train =="; head -n 5 owt_train.txt
!echo "== OWT valid =="; head -n 5 owt_valid.txt

== TinyStories train ==

Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  
He said, “Wow, that is a really amazing vase! Can I buy it?” 
The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. 
== TinyStories valid ==
u don't have to be scared of the loud dog, I'll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.
<|endoftext|>
Once upon a time, i

## 1.3 Training and Validation Files

In [48]:
from pathlib import Path

train_files = [
    "TinyStoriesV2-GPT4-train.txt",
    "owt_train.txt",
]

valid_files = [
    "TinyStoriesV2-GPT4-valid.txt",
    "owt_valid.txt",
]

# Sanity checks
for f in train_files + valid_files:
    p = Path(f)
    assert p.exists(), f"Missing file: {f}"
    assert p.stat().st_size > 0, f"Empty file: {f}"

print("All files found and non-empty.")

All files found and non-empty.


# 2. Text Normalization

## 2.1 Whitespaces

This code reads text line by line and replaces multiple spaces, tabs, or newlines with a single space.


In [49]:
import re

whitespace_pattern = re.compile(r'\s+')

def stream_line(paths):
  for path in paths:
    with open(path, 'r') as f:
      for line in f:
        if line:
          line = line.strip()
          line = whitespace_pattern.sub(' ', line)
          yield line


In [50]:
#test
stream = stream_line(train_files)
for i in range(5):
  print(next(stream))


Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!
He said, “Wow, that is a really amazing vase! Can I buy it?”
The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was.


##2.2 End of Word
Take a word, split it into characters, and append an end-of-word marker.

In [51]:
EOW = "</eow>"
def word_to_symbols(word):
    return tuple(word) + (EOW,)

print(word_to_symbols("hello"))

('h', 'e', 'l', 'l', 'o', '</eow>')


#3. A mechanism to count the frequency of words

In [52]:
from collections import Counter


def build_word_freqs(lines):
  freq = Counter()
  for line in lines:
    for word in line.split(" "):
      symbols = word_to_symbols(word)
      freq[symbols] += 1
  return freq

In [53]:
#test

from itertools import islice

sample_lines = islice(
    stream_line(train_files),
    10_000
)

word_freqs = build_word_freqs(sample_lines)

print("Unique symbol sequences:", len(word_freqs))

for i, (k, v) in enumerate(word_freqs.items()):
    print(k, "→", v)
    if i == 5:
        break


Unique symbol sequences: 11421
('</eow>',) → 703
('O', 'n', 'c', 'e', '</eow>') → 1075
('u', 'p', 'o', 'n', '</eow>') → 1030
('a', '</eow>') → 9703
('t', 'i', 'm', 'e', '</eow>') → 261
('t', 'h', 'e', 'r', 'e', '</eow>') → 1205


#4. Counting Adjacent Symbol Pairs

In [54]:
from collections import Counter

def get_pair_freqs(word_freqs):
    pair_freqs = Counter()

    for symbols, freq in word_freqs.items():
      for i in range(len(symbols) - 1):
        pair = (symbols[i], symbols[i + 1])
        pair_freqs[pair] += freq

    return pair_freqs

In [55]:
pair_freqs = get_pair_freqs(word_freqs)

print("Number of unique symbol pairs:", len(pair_freqs))
pair_freqs.most_common(10)


Number of unique symbol pairs: 951


[(('e', '</eow>'), 49328),
 (('h', 'e'), 40486),
 (('d', '</eow>'), 37565),
 (('t', 'h'), 27762),
 (('.', '</eow>'), 27072),
 (('t', '</eow>'), 21366),
 (('a', 'n'), 20944),
 (('n', 'd'), 20325),
 (('y', '</eow>'), 19459),
 (('s', '</eow>'), 18840)]

#5. Mechanism to merge most frequent pair

In [56]:
best_pair = pair_freqs.most_common(1)[0][0]
best_pair


('e', '</eow>')

In [57]:
def merge_symbols(symbols, pair):
    merged = []
    i = 0
    while i < len(symbols):
        # If we see the target pair, merge it
        if i < len(symbols) - 1 and symbols[i] == pair[0] and symbols[i+1] == pair[1]:
            merged.append(symbols[i] + symbols[i+1])
            i += 2
        else:
            merged.append(symbols[i])
            i += 1
    return tuple(merged)


#6. Apply merge

In [58]:
def apply_merge(word_freqs, pair):
    new_word_freqs = Counter()

    for symbols, freq in word_freqs.items():
        new_symbols = merge_symbols(symbols, pair)
        new_word_freqs[new_symbols] += freq

    return new_word_freqs


In [59]:
word_freqs = apply_merge(word_freqs, best_pair)


In [60]:
for i, (k, v) in enumerate(word_freqs.items()):
    print(k, "→", v)
    if i == 5:
        break


('</eow>',) → 703
('O', 'n', 'c', 'e</eow>') → 1075
('u', 'p', 'o', 'n', '</eow>') → 1030
('a', '</eow>') → 9703
('t', 'i', 'm', 'e</eow>') → 261
('t', 'h', 'e', 'r', 'e</eow>') → 1205


#7. Full Merge Tale

In [61]:
def train_bpe(word_freqs, num_merges=100):
    merges = []  # list of pairs in the order we merged them

    for m in range(num_merges):
        pair_freqs = get_pair_freqs(word_freqs)
        if not pair_freqs:
            break

        best_pair, best_count = pair_freqs.most_common(1)[0]
        merges.append(best_pair)

        word_freqs = apply_merge(word_freqs, best_pair)

        # lightweight progress print
        if (m + 1) % 10 == 0 or m < 5:
            print(f"merge {m+1:4d}: {best_pair}  count={best_count}")

    return merges, word_freqs


In [62]:
#test

from itertools import islice
sample_lines = islice(stream_line(train_files), 10_000)
word_freqs0 = build_word_freqs(sample_lines)

merges, word_freqs_trained = train_bpe(word_freqs0, num_merges=50)

print("Total merges learned:", len(merges))
print("First 10 merges:", merges[:10])

merge    1: ('e', '</eow>')  count=49328
merge    2: ('d', '</eow>')  count=37565
merge    3: ('t', 'h')  count=27762
merge    4: ('.', '</eow>')  count=27072
merge    5: ('t', '</eow>')  count=21366
merge   10: ('t', 'o')  count=14655
merge   20: ('w', 'a')  count=9191
merge   30: ('l', 'l')  count=5755
merge   40: ('e', '.</eow>')  count=3748
merge   50: ('in', 'g</eow>')  count=3386
Total merges learned: 50
First 10 merges: [('e', '</eow>'), ('d', '</eow>'), ('t', 'h'), ('.', '</eow>'), ('t', '</eow>'), ('a', 'n'), ('y', '</eow>'), ('s', '</eow>'), (',', '</eow>'), ('t', 'o')]


#8. Merge Rank Table

In [63]:
merge_ranks = {pair: i for i, pair in enumerate(merges)}


#9. Adjacent Pairs

In [64]:
def adjacent_pairs(symbols):
    return {(symbols[i], symbols[i+1]) for i in range(len(symbols) - 1)}


#10.  BPE - Single word

In [65]:
def bpe_encode_word(word, merge_ranks, eow="</eow>"):
    symbols = tuple(word) + (eow,)

    while True:
        pairs = adjacent_pairs(symbols)
        # find mergeable pairs
        mergeable = [p for p in pairs if p in merge_ranks]
        if not mergeable:
            break

        # pick the pair with best (lowest) rank
        best = min(mergeable, key=lambda p: merge_ranks[p])
        symbols = merge_symbols(symbols, best)

    return symbols


In [66]:
def bpe_encode_text(text, merge_ranks, lowercase=False):
    if lowercase:
        text = text.lower()
    words = text.strip().split()
    out = []
    for w in words:
        out.extend(bpe_encode_word(w, merge_ranks))
    return out


In [67]:
tests = [
    "Once upon a time there was a cat",
    "the the the",
    "there was time",
]

for t in tests:
    print("TEXT:", t)
    print("TOKENS:", bpe_encode_text(t, merge_ranks))
    print()


TEXT: Once upon a time there was a cat
TOKENS: ['O', 'n', 'c', 'e</eow>', 'u', 'p', 'on', '</eow>', 'a</eow>', 't', 'i', 'm', 'e</eow>', 'th', 'er', 'e</eow>', 'was</eow>', 'a</eow>', 'c', 'a', 't</eow>']

TEXT: the the the
TOKENS: ['the</eow>', 'the</eow>', 'the</eow>']

TEXT: there was time
TOKENS: ['th', 'er', 'e</eow>', 'was</eow>', 't', 'i', 'm', 'e</eow>']



# 11. Vocab building plus, conversion to integer ID

In [68]:
def build_token_set(word_freqs):
    tokens = set()
    for symbols in word_freqs.keys():
        for s in symbols:
            tokens.add(s)
    return tokens

tokens = build_token_set(word_freqs_trained)

print("Number of tokens in vocab (from trained sample):", len(tokens))
print("Example tokens:", list(sorted(tokens))[:50])


Number of tokens in vocab (from trained sample): 125
Example tokens: ['!', '"', '"</eow>', "'", ',', ',</eow>', '-', '.', '.</eow>', '0', '1', '3', '8', '9', ':', ';', '<', '</eow>', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'The', 'The</eow>', 'They</eow>', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a']


In [69]:
SPECIAL_TOKENS = ["<pad>", "<unk>", "<bos>", "<eos>"]

for t in SPECIAL_TOKENS:
    tokens.add(t)


In [70]:
tokens_sorted = sorted(tokens)

token_to_id = {t: i for i, t in enumerate(tokens_sorted)}
id_to_token = {i: t for t, i in token_to_id.items()}

print("Vocab size:", len(token_to_id))
print("IDs:", {t: token_to_id[t] for t in SPECIAL_TOKENS})


Vocab size: 129
IDs: {'<pad>': 20, '<unk>': 21, '<bos>': 18, '<eos>': 19}


In [71]:
UNK_ID = token_to_id["<unk>"]

def encode_to_ids(text, merge_ranks, token_to_id):
    toks = bpe_encode_text(text, merge_ranks)
    return [token_to_id.get(tok, UNK_ID) for tok in toks]


In [72]:
s = "Once upon a time there was a cat"
print(encode_to_ids(s, merge_ranks, token_to_id)[:50])


[38, 91, 60, 66, 112, 97, 94, 17, 54, 105, 79, 89, 66, 107, 69, 66, 117, 54, 60, 53, 106]


In [73]:
import json

# Save merges in standard “pair per line” format
with open("merges.txt", "w", encoding="utf-8") as f:
    for a, b in merges:
        f.write(f"{a} {b}\n")

# Save vocab as JSON
with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(token_to_id, f, ensure_ascii=False, indent=2)

print("Saved merges.txt and vocab.json")


Saved merges.txt and vocab.json


In [74]:
with open("vocab.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)

# View first 30 tokens
for i, (k, v) in enumerate(vocab.items()):
    print(k, "→", v)
    if i == 100:
        break

! → 0
" → 1
"</eow> → 2
' → 3
, → 4
,</eow> → 5
- → 6
. → 7
.</eow> → 8
0 → 9
1 → 10
3 → 11
8 → 12
9 → 13
: → 14
; → 15
< → 16
</eow> → 17
<bos> → 18
<eos> → 19
<pad> → 20
<unk> → 21
> → 22
? → 23
A → 24
B → 25
C → 26
D → 27
E → 28
F → 29
G → 30
H → 31
I → 32
J → 33
K → 34
L → 35
M → 36
N → 37
O → 38
P → 39
Q → 40
R → 41
S → 42
T → 43
The → 44
The</eow> → 45
They</eow> → 46
U → 47
V → 48
W → 49
X → 50
Y → 51
Z → 52
a → 53
a</eow> → 54
an → 55
and</eow> → 56
ar → 57
b → 58
bi → 59
c → 60
d → 61
d</eow> → 62
do → 63
e → 64
e.</eow> → 65
e</eow> → 66
ed</eow> → 67
en → 68
er → 69
er</eow> → 70
f → 71
g → 72
g</eow> → 73
h → 74
ha → 75
he → 76
he</eow> → 77
hi → 78
i → 79
id → 80
in → 81
ing</eow> → 82
j → 83
k → 84
l → 85
la → 86
li → 87
ll → 88
m → 89
m</eow> → 90
n → 91
o → 92
om → 93
on → 94
or → 95
ou → 96
p → 97
q → 98
r → 99
re → 100


# 12. Decoder

In [75]:
def decode_ids(ids, id_to_token, eow="</eow>"):

    skip = {"<pad>", "<bos>", "<eos>"}  # keep <unk> visible unless you prefer skipping it

    tokens = []
    for i in ids:
        tok = id_to_token.get(i, "<unk>")
        if tok in skip:
            continue
        tokens.append(tok)

    # Build text by concatenating pieces; whenever we see eow, we emit a space.
    out_chars = []
    for tok in tokens:
        if tok == eow:
            out_chars.append(" ")
            continue

        if tok.endswith(eow):
            # token like 'e</w>' or 'there</w>'
            out_chars.append(tok[: -len(eow)])
            out_chars.append(" ")
        else:
            out_chars.append(tok)

    text = "".join(out_chars).strip()
    return text


In [76]:
s = "Once upon a time there was a cat"
ids = encode_to_ids(s, merge_ranks, token_to_id)
print("IDS:", ids[:30])

decoded = decode_ids(ids, id_to_token)
print("DECODED:", decoded)


IDS: [38, 91, 60, 66, 112, 97, 94, 17, 54, 105, 79, 89, 66, 107, 69, 66, 117, 54, 60, 53, 106]
DECODED: Once upon a time there was a cat


#